In [18]:
q= '''
SELECT *
FROM
    (SELECT *,
    row_number()
    OVER
        (PARTITION BY "open_time"
        ORDER BY "close_time" DESC, Api_Invocation_Time DESC, write_time DESC)
    AS row_number
    FROM "sagemaker_featurestore"."btcpricedata_1704246894")
WHERE row_number = 1 ORDER BY close_time ASC;
'''

In [19]:
from sagemaker.feature_store.feature_group import FeatureGroup
import boto3
import pandas as pd
import numpy as np
import io
from sagemaker.session import Session
from sagemaker import get_execution_role
sagemaker_session = Session()

feature_group = FeatureGroup(name = 'BtcPriceData')

query = feature_group.athena_query()

table = query.table_name
table
default_s3_bucket_name = 'binance-btc-prices'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [20]:

# run Athena query. The output is loaded to a Pandas dataframe.
dataset = pd.DataFrame()
query.run(query_string=q, output_location='s3://'+default_s3_bucket_name+'/query_results/')
query.wait()
dataset = query.as_dataframe()

In [21]:
dataset['open_time'] = pd.to_datetime(dataset['open_time'], unit='ms').dt.strftime('%Y-%m-%d %T')
dataset

,open_time,high_price,low_price,close_price,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,open_price,write_time,api_invocation_time,is_deleted,row_number
0,2019-09-09 00:00:00,10392.41,10376.45,10390.08,27.884150,1.567987e+09,2.896169e+05,626.0,12.492958,10381.24,2024-01-08 18:11:09.934,2024-01-08 18:05:58.000,False,1
1,2019-09-09 00:05:00,10393.94,10368.87,10372.31,36.750216,1.567988e+09,3.814708e+05,614.0,22.195335,10390.74,2024-01-08 18:11:07.429,2024-01-08 18:05:58.000,False,1
2,2019-09-09 00:10:00,10386.24,10368.88,10374.20,38.888045,1.567988e+09,4.035607e+05,640.0,19.145836,10372.31,2024-01-08 18:11:30.173,2024-01-08 18:05:58.000,False,1
3,2019-09-09 00:15:00,10387.83,10368.89,10384.89,24.997683,1.567988e+09,2.593937e+05,712.0,11.799247,10374.18,2024-01-08 18:11:10.880,2024-01-08 18:05:58.000,False,1
4,2019-09-09 00:20:00,10396.00,10382.40,10394.20,18.551736,1.567989e+09,1.927592e+05,571.0,9.729698,10386.25,2024-01-08 18:11:06.932,2024-01-08 18:05:58.000,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452547,2024-01-07 23:35:00,43874.70,43732.45,43759.70,237.834260,1.704671e+09,1.041494e+07,12630.0,141.030650,43732.45,2024-01-08 19:07:48.199,2024-01-08 19:07:13.000,False,1
452548,2024-01-07 23:40:00,43774.65,43572.09,43619.99,386.292560,1.704671e+09,1.687144e+07,12556.0,139.491970,43757.91,2024-01-08 19:07:45.397,2024-01-08 19:07:13.000,False,1
452549,2024-01-07 23:45:00,43775.78,43619.99,43773.54,164.609100,1.704671e+09,7.192140e+06,6625.0,84.019760,43620.00,2024-01-08 19:07:45.397,2024-01-08 19:07:13.000,False,1
452550,2024-01-07 23:50:00,43863.44,43751.67,43858.24,145.929780,1.704672e+09,6.393986e+06,19874.0,95.597510,43773.54,2024-01-08 19:07:50.001,2024-01-08 19:07:13.000,False,1


In [22]:
dataset.to_csv('btc_prices.csv')